# Figura 4: Comparacion Modo Posterior vs Media MCMC

Este notebook compara los resultados de la descomposicion historica usando:
- **Modo Posterior** (`mh_replic=0`): Rapido, usa un solo punto
- **Media Posterior MCMC** (`mh_replic=10000`): Lento, promedia sobre distribucion

**Objetivo**: Verificar si la media posterior MCMC da resultados mas cercanos al paper original de S&W (2007).

**ADVERTENCIA**: La ejecucion con MCMC tarda ~30-60 minutos.

## 1. Setup

In [1]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
from pathlib import Path
import time

# Add parent directory to path
sys.path.append(str(Path.cwd().parent.parent))

from direct_replication import DynareInterface

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("Imports completados")

Imports completados


In [2]:
# Configurar rutas
os.environ['OCTAVE_EXECUTABLE'] = r'C:\Program Files\GNU Octave\Octave-10.3.0\mingw64\bin\octave-cli.exe'

DYNARE_PATH = r'C:\dynare\6.5\matlab'
MODEL_PATH = Path.cwd().parent / 'model'

# Configuracion
START_YEAR = 1966
START_QUARTER = 1

SHOCK_LABELS = {
    'ea': 'Productivity', 'eb': 'Risk premium', 'eg': 'Exog. spending',
    'eqs': 'Investment', 'em': 'Monetary', 'epinf': 'Price markup', 'ew': 'Wage markup'
}
SHOCK_ORDER = ['ea', 'eb', 'eg', 'eqs', 'em', 'epinf', 'ew']

print(f"Model path: {MODEL_PATH}")

Model path: c:\Users\HP\OneDrive\Escritorio\David Guzzi\Github\MECTMT11\direct_replication\model


## 2. Funcion de Extraccion

In [3]:
def extract_shock_decomposition(di, variables=['dy', 'pinfobs']):
    """
    Extrae la descomposicion historica de shocks de Dynare.
    """
    has_decomp = di.oc.eval('isfield(oo_, "shock_decomposition")', nout=1)
    if not has_decomp:
        raise RuntimeError("Shock decomposition no encontrada.")
    
    decomp_size = di.oc.eval('size(oo_.shock_decomposition)', nout=1)
    decomp_size = np.array(decomp_size).flatten()
    n_vars = int(decomp_size[0])
    n_cols = int(decomp_size[1])
    n_periods = int(decomp_size[2])
    n_shocks = n_cols - 2
    
    endo_names_raw = di.oc.eval('M_.endo_names', nout=1)
    if hasattr(endo_names_raw, 'tolist'):
        endo_names = [str(n).strip() for n in endo_names_raw.flatten()]
    else:
        endo_names = [str(n).strip() for n in endo_names_raw]
    
    shock_names = ['ea', 'eb', 'eg', 'eqs', 'em', 'epinf', 'ew']
    
    var_indices = {}
    for var in variables:
        try:
            idx = endo_names.index(var)
            var_indices[var] = idx
        except ValueError:
            print(f"WARNING: Variable '{var}' no encontrada")
    
    decomp_array = di.oc.eval('oo_.shock_decomposition', nout=1)
    
    data = []
    for var, v_idx in var_indices.items():
        for t in range(n_periods):
            total_quarters = (START_QUARTER - 1) + t
            year = START_YEAR + total_quarters // 4
            quarter = (total_quarters % 4) + 1
            
            for s_idx, shock in enumerate(shock_names):
                data.append({
                    'variable': var, 'shock': shock, 'period': t + 1,
                    'year': year, 'quarter': quarter,
                    'contribution': float(decomp_array[v_idx, s_idx, t])
                })
            
            data.append({
                'variable': var, 'shock': 'initial', 'period': t + 1,
                'year': year, 'quarter': quarter,
                'contribution': float(decomp_array[v_idx, n_shocks, t])
            })
            
            data.append({
                'variable': var, 'shock': 'smoothed', 'period': t + 1,
                'year': year, 'quarter': quarter,
                'contribution': float(decomp_array[v_idx, n_shocks + 1, t])
            })
    
    return pd.DataFrame(data)

def compute_annual(df, variable):
    """Convierte datos trimestrales a anuales."""
    var_data = df[df['variable'] == variable].copy()
    if variable == 'dy':
        annual = var_data.groupby(['year', 'shock']).agg({'contribution': 'sum'}).reset_index()
    else:
        annual = var_data.groupby(['year', 'shock']).agg({'contribution': 'mean'}).reset_index()
    annual['variable'] = variable
    return annual

print("Funciones definidas")

Funciones definidas


## 3. Cargar Resultados del Modo Posterior (ya calculados)

In [4]:
# Cargar datos del modo posterior (mh_replic=0) ya calculados
output_dir = Path.cwd()

mode_gdp = pd.read_csv(output_dir / 'figure4_gdp_annual_decomposition.csv')
mode_inf = pd.read_csv(output_dir / 'figure4_inflation_annual_decomposition.csv')

print(f"Datos modo posterior cargados:")
print(f"  GDP: {len(mode_gdp)} filas, anos {mode_gdp['year'].min()}-{mode_gdp['year'].max()}")
print(f"  Inflation: {len(mode_inf)} filas")

Datos modo posterior cargados:
  GDP: 360 filas, anos 1966-2005
  Inflation: 360 filas


## 4. Ejecutar Modelo con MCMC

**ADVERTENCIA**: Esta celda tarda ~30-60 minutos en ejecutar.

In [5]:
# Inicializar Dynare
di = DynareInterface(DYNARE_PATH, str(MODEL_PATH))
print("Interfaz Dynare inicializada")

    _pyeval at line 57 column 10

    _pyeval at line 57 column 10

    _pyeval at line 57 column 10

Interfaz Dynare inicializada


In [ ]:
# Ejecutar modelo con MCMC
print("="*70)
print("EJECUTANDO MODELO CON MCMC (mh_replic=10000)")
print("ADVERTENCIA: Esto tardara ~30-60 minutos")
print("="*70)

start_time = time.time()

di.run_model('usmodel_figure4_mcmc.mod')

elapsed = time.time() - start_time
print(f"\nTiempo de ejecucion: {elapsed/60:.1f} minutos")

EJECUTANDO MODELO CON MCMC (mh_replic=10000)
ADVERTENCIA: Esto tardara ~30-60 minutos

Step 1: Closing Octave session to release file locks...
Waiting for Windows to release file handles...

Step 2: Cleaning up directories...
Searching for directories to clean up...
No directories found to clean up.

Step 3: Starting fresh Octave session...
    _pyeval at line 57 column 10

    _pyeval at line 57 column 10

    _pyeval at line 57 column 10

✓ Octave session ready

Step 4: Running Dynare estimation...
Command: dynare usmodel_figure4_mcmc nograph
(This may take several minutes...)

Starting Dynare (version 6.5).
Calling Dynare with arguments: nograph
Starting preprocessing of the model file ...
Found 40 equation(s).
Evaluating expressions...
Computing static model derivatives (order 1).
Normalizing the static model...
Finding the optimal block decomposition of the static model...
11 block(s) found:
  9 recursive block(s) and 2 simultaneous block(s).
  the largest simultaneous block has 1

In [ ]:
# Extraer shock decomposition de MCMC
print("Extrayendo shock decomposition de MCMC...")
mcmc_decomp = extract_shock_decomposition(di, ['dy', 'pinfobs'])

# Convertir a anual
mcmc_gdp = compute_annual(mcmc_decomp, 'dy')
mcmc_inf = compute_annual(mcmc_decomp, 'pinfobs')

print(f"Datos MCMC extraidos:")
print(f"  GDP: {len(mcmc_gdp)} filas")
print(f"  Inflation: {len(mcmc_inf)} filas")

## 5. Comparar Resultados

In [ ]:
# Comparar anos clave
key_years = [1975, 1980, 1982, 1991, 2001]

print("="*70)
print("COMPARACION: Modo Posterior vs Media MCMC")
print("="*70)

for year in key_years:
    print(f"\n{year}:")
    print("-" * 50)
    
    # GDP Growth
    mode_val = mode_gdp[(mode_gdp['year'] == year) & (mode_gdp['shock'] == 'smoothed')]['contribution'].values
    mcmc_val = mcmc_gdp[(mcmc_gdp['year'] == year) & (mcmc_gdp['shock'] == 'smoothed')]['contribution'].values
    
    if len(mode_val) > 0 and len(mcmc_val) > 0:
        diff = mcmc_val[0] - mode_val[0]
        print(f"  GDP Growth: Mode={mode_val[0]:+.2f}%, MCMC={mcmc_val[0]:+.2f}%, Diff={diff:+.2f}")
    
    # Inflation
    mode_val = mode_inf[(mode_inf['year'] == year) & (mode_inf['shock'] == 'smoothed')]['contribution'].values
    mcmc_val = mcmc_inf[(mcmc_inf['year'] == year) & (mcmc_inf['shock'] == 'smoothed')]['contribution'].values
    
    if len(mode_val) > 0 and len(mcmc_val) > 0:
        diff = mcmc_val[0] - mode_val[0]
        print(f"  Inflation:  Mode={mode_val[0]:+.2f}%, MCMC={mcmc_val[0]:+.2f}%, Diff={diff:+.2f}")

In [ ]:
# Comparar contribucion de shocks por ano clave
print("\n" + "="*70)
print("DETALLE: Contribucion de shocks en 1975 (Oil Crisis)")
print("="*70)

year = 1975
print("\nGDP Growth:")
print(f"{'Shock':<15} {'Mode':>10} {'MCMC':>10} {'Diff':>10}")
print("-" * 45)

for shock in SHOCK_ORDER:
    mode_val = mode_gdp[(mode_gdp['year'] == year) & (mode_gdp['shock'] == shock)]['contribution'].values
    mcmc_val = mcmc_gdp[(mcmc_gdp['year'] == year) & (mcmc_gdp['shock'] == shock)]['contribution'].values
    
    if len(mode_val) > 0 and len(mcmc_val) > 0:
        diff = mcmc_val[0] - mode_val[0]
        print(f"{SHOCK_LABELS[shock]:<15} {mode_val[0]:>+10.2f} {mcmc_val[0]:>+10.2f} {diff:>+10.2f}")

## 6. Grafico Comparativo

In [ ]:
# Grafico comparativo: Modo vs MCMC
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

years = sorted([y for y in mode_gdp['year'].unique() if y <= 2004])

# Panel 1: GDP Growth - Smoothed values
ax = axes[0, 0]
mode_smooth = mode_gdp[mode_gdp['shock'] == 'smoothed'].sort_values('year')
mcmc_smooth = mcmc_gdp[mcmc_gdp['shock'] == 'smoothed'].sort_values('year')
mode_smooth = mode_smooth[mode_smooth['year'] <= 2004]
mcmc_smooth = mcmc_smooth[mcmc_smooth['year'] <= 2004]

ax.plot(mode_smooth['year'], mode_smooth['contribution'], 'b-o', label='Modo Posterior', markersize=4)
ax.plot(mcmc_smooth['year'], mcmc_smooth['contribution'], 'r-s', label='Media MCMC', markersize=4)
ax.axhline(0, color='black', linewidth=0.5)
ax.set_title('GDP Growth: Valor Suavizado', fontweight='bold')
ax.set_ylabel('Percent')
ax.legend()
ax.grid(True, alpha=0.3)

# Panel 2: Inflation - Smoothed values
ax = axes[0, 1]
mode_smooth = mode_inf[mode_inf['shock'] == 'smoothed'].sort_values('year')
mcmc_smooth = mcmc_inf[mcmc_inf['shock'] == 'smoothed'].sort_values('year')
mode_smooth = mode_smooth[mode_smooth['year'] <= 2004]
mcmc_smooth = mcmc_smooth[mcmc_smooth['year'] <= 2004]

ax.plot(mode_smooth['year'], mode_smooth['contribution'], 'b-o', label='Modo Posterior', markersize=4)
ax.plot(mcmc_smooth['year'], mcmc_smooth['contribution'], 'r-s', label='Media MCMC', markersize=4)
ax.axhline(0, color='black', linewidth=0.5)
ax.set_title('Inflation: Valor Suavizado', fontweight='bold')
ax.set_ylabel('Percent')
ax.legend()
ax.grid(True, alpha=0.3)

# Panel 3: GDP Growth - Diferencia
ax = axes[1, 0]
mode_smooth = mode_gdp[mode_gdp['shock'] == 'smoothed'].sort_values('year')
mcmc_smooth = mcmc_gdp[mcmc_gdp['shock'] == 'smoothed'].sort_values('year')
mode_smooth = mode_smooth[mode_smooth['year'] <= 2004]
mcmc_smooth = mcmc_smooth[mcmc_smooth['year'] <= 2004]

if len(mode_smooth) == len(mcmc_smooth):
    diff = mcmc_smooth['contribution'].values - mode_smooth['contribution'].values
    ax.bar(mode_smooth['year'], diff, color='purple', alpha=0.7)
ax.axhline(0, color='black', linewidth=0.5)
ax.set_title('GDP Growth: Diferencia (MCMC - Mode)', fontweight='bold')
ax.set_ylabel('Percent')
ax.set_xlabel('Year')
ax.grid(True, alpha=0.3)

# Panel 4: Inflation - Diferencia
ax = axes[1, 1]
mode_smooth = mode_inf[mode_inf['shock'] == 'smoothed'].sort_values('year')
mcmc_smooth = mcmc_inf[mcmc_inf['shock'] == 'smoothed'].sort_values('year')
mode_smooth = mode_smooth[mode_smooth['year'] <= 2004]
mcmc_smooth = mcmc_smooth[mcmc_smooth['year'] <= 2004]

if len(mode_smooth) == len(mcmc_smooth):
    diff = mcmc_smooth['contribution'].values - mode_smooth['contribution'].values
    ax.bar(mode_smooth['year'], diff, color='purple', alpha=0.7)
ax.axhline(0, color='black', linewidth=0.5)
ax.set_title('Inflation: Diferencia (MCMC - Mode)', fontweight='bold')
ax.set_ylabel('Percent')
ax.set_xlabel('Year')
ax.grid(True, alpha=0.3)

plt.suptitle('Comparacion: Modo Posterior vs Media MCMC\n(Figure 4 Historical Decomposition)', 
             fontsize=14, fontweight='bold')
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

## 7. Guardar Resultados MCMC

In [ ]:
# Guardar datos MCMC
mcmc_gdp.to_csv(output_dir / 'figure4_gdp_mcmc_decomposition.csv', index=False)
mcmc_inf.to_csv(output_dir / 'figure4_inflation_mcmc_decomposition.csv', index=False)

print("Datos MCMC guardados:")
print(f"  - figure4_gdp_mcmc_decomposition.csv")
print(f"  - figure4_inflation_mcmc_decomposition.csv")

## 8. Conclusiones

In [ ]:
# Calcular estadisticas de diferencia
mode_smooth_gdp = mode_gdp[(mode_gdp['shock'] == 'smoothed') & (mode_gdp['year'] <= 2004)].sort_values('year')
mcmc_smooth_gdp = mcmc_gdp[(mcmc_gdp['shock'] == 'smoothed') & (mcmc_gdp['year'] <= 2004)].sort_values('year')

mode_smooth_inf = mode_inf[(mode_inf['shock'] == 'smoothed') & (mode_inf['year'] <= 2004)].sort_values('year')
mcmc_smooth_inf = mcmc_inf[(mcmc_inf['shock'] == 'smoothed') & (mcmc_inf['year'] <= 2004)].sort_values('year')

if len(mode_smooth_gdp) == len(mcmc_smooth_gdp):
    diff_gdp = mcmc_smooth_gdp['contribution'].values - mode_smooth_gdp['contribution'].values
    diff_inf = mcmc_smooth_inf['contribution'].values - mode_smooth_inf['contribution'].values
    
    print("="*70)
    print("CONCLUSIONES")
    print("="*70)
    print("\nDiferencia entre Media MCMC y Modo Posterior:")
    print(f"\n  GDP Growth:")
    print(f"    Mean diff: {np.mean(diff_gdp):+.3f}")
    print(f"    Max diff:  {np.max(np.abs(diff_gdp)):+.3f}")
    print(f"    Std diff:  {np.std(diff_gdp):.3f}")
    
    print(f"\n  Inflation:")
    print(f"    Mean diff: {np.mean(diff_inf):+.3f}")
    print(f"    Max diff:  {np.max(np.abs(diff_inf)):+.3f}")
    print(f"    Std diff:  {np.std(diff_inf):.3f}")
    
    print("\n" + "-"*50)
    if np.max(np.abs(diff_gdp)) < 0.5 and np.max(np.abs(diff_inf)) < 0.5:
        print("Las diferencias son PEQUENAS (<0.5%).")
        print("La diferencia con el paper NO se debe a usar modo vs media MCMC.")
    else:
        print("Las diferencias son SIGNIFICATIVAS.")
        print("Usar media MCMC puede acercarnos mas al paper original.")

## 9. Cleanup

In [ ]:
di.close()
print("Sesion Octave cerrada")